# Segmentation + Rule Discovery (Bank Marketing)

Mục tiêu:
1) Segment khách hàng dựa trên đặc trưng khách hàng + hành vi campaign (KHÔNG dùng y).
2) Profile từng segment (chân dung segment).
3) Đánh giá hiệu quả campaign theo segment.
4) Khai thác luật & pattern theo từng segment bằng Decision Tree (rule extraction).

Lưu ý:
- `y` chỉ dùng để đánh giá/giải thích sau khi đã phân cụm.
- `duration` có thể gây "leakage": nó phản ánh cuộc gọi đã diễn ra (hậu quả), nên luật dùng duration cần diễn giải đúng (proxy mức quan tâm), không phải biến để ra quyết định gọi trước.


In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/processed/bank_marketing_raw.csv")

In [3]:
df["y"] = df["y"].astype(str).str.lower().map({"yes": 1, "no": 0})
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,...,duration,campaign,pdays,previous,poutcome,y,poutcome_missing,target,pdays_contacted,has_previous_campaign
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,...,261,1,-1,0,no_previous_campaign,0,1,0,1,0
1,44,technician,single,secondary,no,29,yes,no,NaN,5,...,151,1,-1,0,no_previous_campaign,0,1,0,1,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,...,76,1,-1,0,no_previous_campaign,0,1,0,1,0
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,...,92,1,-1,0,no_previous_campaign,0,1,0,1,0
4,33,NaN,single,NaN,no,1,no,no,NaN,5,...,198,1,-1,0,no_previous_campaign,0,1,0,1,0


segment dựa trên câu hỏi: “Khách hàng là ai + họ đã tương tác với campaign như thế nào”

In [4]:
NUM_COLS = ["age", "campaign", "previous", "duration"]
CAT_COLS = ["job", "marital", "education", "contact"]

# clean NA cơ bản
for c in CAT_COLS:
    df[c] = df[c].fillna("unknown").astype(str)

for c in NUM_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")
    df[c] = df[c].fillna(df[c].median())

df[NUM_COLS + CAT_COLS + ["y"]].head()


,age,campaign,previous,duration,job,marital,education,contact,y
0,58,1,0,261,management,married,tertiary,unknown,0
1,44,1,0,151,technician,single,secondary,unknown,0
2,33,1,0,76,entrepreneur,married,secondary,unknown,0
3,47,1,0,92,blue-collar,married,unknown,unknown,0
4,33,1,0,198,unknown,single,unknown,unknown,0


“Các biến được chọn cho segmentation bao gồm đặc điểm nhân khẩu học và hành vi tương tác với campaign, nhằm phản ánh cả ‘ai là khách hàng’ và ‘họ phản ứng với tiếp thị như thế nào’, trong khi biến mục tiêu y được loại bỏ để tránh rò rỉ thông tin.”

Dữ liệu lệch + phân phối không đều --> chuẩn hoá

### Feature Engineering cho Segmentation

##### 1. Vấn đề của dữ liệu gốc
Dữ liệu Bank Marketing có các đặc điểm sau:

- **Biến số (numeric)** như `age`, `campaign`, `previous`, `duration`:
  - Phân phối **không đều** (skewed), đặc biệt là `campaign`, `previous`, `duration`
  - Thang đo rất khác nhau (ví dụ: `age` ~ vài chục, `duration` ~ vài trăm/ nghìn giây)
  - Nếu không chuẩn hoá, các biến có giá trị lớn sẽ **chi phối khoảng cách Euclidean**

- **Biến phân loại (categorical)** như `job`, `marital`, `education`, `contact`:
  - Không thể đưa trực tiếp vào các thuật toán clustering
  - Có nhiều category, trong đó tồn tại các giá trị `"unknown"`





In [5]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), NUM_COLS),
        ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_COLS),
    ],
    remainder="drop",
    sparse_threshold=0.3
)

X = preprocess.fit_transform(df)
X.shape
    

(45211, 26)

### PCA + Clustering (MiniBatchKMeans & HDBSCAN) + Rule Extraction theo Segment

Pipeline:
1) Tiền xử lý (numeric scale + categorical one-hot)
2) PCA giảm chiều để:
   - giảm nhiễu
   - tăng tốc clustering
3) Phân cụm:
   - MiniBatchKMeans (cần chọn k)
   - HDBSCAN (tự tìm số cụm + có noise = -1)
4) Profiling & đánh giá bằng business signal (conversion, campaign pattern)
5) Rule extraction theo từng segment bằng Decision Tree


In [6]:
from sklearn.decomposition import PCA


pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X)

X.shape, X_pca.shape, pca.explained_variance_ratio_.sum()
#PCA do nhiêu chiều

((45211, 26), (45211, 14), 0.9607285300227953)

#### MiniBatchKMeans

In [7]:
from sklearn.cluster import MiniBatchKMeans


def pick_k_mbkmeans(X_pca, k_range=range(2, 9), sample_size=6000, seed=42):
    rng = np.random.default_rng(seed)
    idx = rng.choice(X_pca.shape[0], size=min(sample_size, X_pca.shape[0]), replace=False)
    Xs = X_pca[idx]

    rows = []
    for k in k_range:
        mbk = MiniBatchKMeans(
            n_clusters=k,
            batch_size=2048,
            n_init=10,
            random_state=seed
        )
        labels = mbk.fit_predict(X_pca)  # fit full để có nhãn đầy đủ
        sil = silhouette_score(Xs, labels[idx])  # silhouette trên sample
        rows.append((k, sil))
    return pd.DataFrame(rows, columns=["k", "silhouette_sample"]).sort_values("silhouette_sample", ascending=False)

k_table = pick_k_mbkmeans(X_pca, k_range=range(2, 9))
k_table


,k,silhouette_sample
0,2,0.147035
6,8,0.132769
2,4,0.128223
3,5,0.125429
4,6,0.116033
5,7,0.111015
1,3,0.099522


In [9]:
BEST_K = int(k_table.iloc[0]["k"])  # hoặc tự set như 4/5 cho dễ business
BEST_K

2

- Với dữ liệu hành vi khách hàng có phân phối chồng lấn:
  - Silhouette từ **0.10–0.25** đã được xem là chấp nhận được

Do đó, các giá trị silhouette trong bảng không phải là “xấu”, mà phản ánh **bản chất khó tách rạch ròi của dữ liệu**.

In [10]:
mbk = MiniBatchKMeans(
    n_clusters=BEST_K,
    batch_size=2048,
    n_init=10,
    random_state=42
)
df["segment_mbk"] = mbk.fit_predict(X_pca)

df["segment_mbk"].value_counts().sort_index()


segment_mbk
0    27747
1    17464
Name: count, dtype: int64

k = 2 thường tương ứng với:

Nhóm khách hàng phản hồi thấp / ít quan tâm

Nhóm khách hàng phản hồi tốt / có mức độ quan tâm cao

Đây là cấu trúc “coarse segmentation”, phù hợp cho bước khám phá ban đầu

Lưu ý: Việc chọn k = 2 không có nghĩa là chỉ tồn tại 2 loại khách hàng, mà chỉ phản ánh hai hành vi chi phối mạnh nhất trong dữ liệu.

####  HDBSCAN

In [25]:
#Thống kê số cụm & noise rate
import hdbscan

hdb = hdbscan.HDBSCAN(
    min_cluster_size=800,
    min_samples=20,
    metric="euclidean"
)

df["segment_hdb"] = hdb.fit_predict(X_pca)

df["segment_hdb"].value_counts().head(10), df["segment_hdb"].unique()[:10]



(segment_hdb
 -1     8783
  13    3472
  2     3039
  3     2810
  12    2664
  8     2508
  16    2373
  9     1952
  14    1780
  18    1591
 Name: count, dtype: int64,
 array([ 4,  2,  8,  5,  1,  7,  6, -1,  9, 15]))

In [26]:
n_noise = (df["segment_hdb"] == -1).sum()
noise_rate = n_noise / len(df)

n_clusters = df.loc[df["segment_hdb"] != -1, "segment_hdb"].nunique()

n_clusters, noise_rate


(21, 0.1942668819535069)

-Đây là nhóm khách hàng:

-Phản ứng rất khác nhau

-Không có pattern rõ ràng

-Không nên ép nhóm này vào segment cụ thể

### Profiling segment

Ta không chỉ nhìn silhouette.
Ta kiểm tra:
- conversion rate theo segment
- hành vi campaign theo segment
- duration/previous khác biệt không

Cluster tốt khi:
- conversion khác biệt rõ giữa segment
- pattern campaign rõ rệt


In [14]:
def profile_segments(df, seg_col):
    out = (
        df.groupby(seg_col)
          .agg(
              n=("y", "size"),
              conversion=("y", "mean"),
              age_mean=("age", "mean"),
              campaign_mean=("campaign", "mean"),
              previous_mean=("previous", "mean"),
              duration_mean=("duration", "mean"),
          )
          .sort_values("conversion", ascending=False)
    )
    return out

profile_mbk = profile_segments(df, "segment_mbk")
profile_hdb = profile_segments(df[df["segment_hdb"] != -1], "segment_hdb")  # bỏ noise cho dễ nhìn

profile_mbk, profile_hdb.head(10)


(                 n  conversion   age_mean  campaign_mean  previous_mean  \
 segment_mbk                                                               
 0            27747    0.120121  33.958842       2.689264       0.628104   
 1            17464    0.112002  52.021931       2.882329       0.504409   
 
              duration_mean  
 segment_mbk                 
 0               266.151836  
 1               245.470454  ,
                 n  conversion   age_mean  campaign_mean  previous_mean  \
 segment_hdb                                                              
 16            486    0.201646  29.946502       1.681070       0.183128   
 11           1755    0.169231  33.027350       2.156695       0.376638   
 17            831    0.168472  29.401925       1.630566       0.276775   
 13            432    0.129630  32.039352       2.025463       0.210648   
 15            597    0.120603  32.016750       1.659966       0.371859   
 18           3122    0.115631  41.520179       

profiling kết quả phân cụm, tức là chân dung trung bình của từng segment.
Mỗi dòng = 1 segment Mỗi cột = đặc trưng trung bình của segment đó

n: số khách hàng trong segment

conversion: tỷ lệ đăng ký thành công (y=1)

age_mean: tuổi trung bình

campaign_mean: số lần gọi trung bình trong campaign hiện tại

previous_mean: số lần đã được gọi ở các campaign trước

duration_mean: thời lượng cuộc gọi trung bình (giây)

### Phân tích & gọi tên các segment?????

## 2. MiniBatchKMeans (k = 2) – Segmentation tổng quát

| Segment | Quy mô | Conversion | Đặc trưng chính | Tên Business |
|-------|--------|------------|-----------------|--------------|
| 0 | 27,747 | 12.0% | Trẻ hơn, thời lượng gọi dài hơn | **Engaged Younger Clients** |
| 1 | 17,464 | 11.2% | Lớn tuổi hơn, tương tác thấp hơn | **Less-engaged Older Clients** |

**Diễn giải:**
- Dữ liệu chủ yếu chia thành hai nhóm hành vi lớn: nhóm khách trẻ, cởi mở hơn và nhóm khách lớn tuổi, ít tương tác hơn.
- Tuy nhiên, mức chênh lệch conversion giữa hai nhóm không lớn → khó đưa ra chiến lược cụ thể.

MiniBatchKMeans phù hợp cho **overview**, không tối ưu cho khai thác luật chi tiết.

---

## 3. HDBSCAN – Segmentation chi tiết & hành động được

### 3.1. Các segment có conversion CAO (ưu tiên)

| Segment | Conversion | Đặc trưng | Tên Business |
|-------|------------|-----------|--------------|
| 16 | **20.1%** | Rất trẻ, ít cuộc gọi, phản hồi tốt | **High-Potential Young Responders** |
| 11 | 16.9% | Trẻ, có lịch sử tương tác | **Returning Interested Clients** |
| 17 | 16.8% | Rất trẻ, gọi ít, hiệu quả cao | **Quick-Decision Young Clients** |

**Insight chung:**
- Khách trẻ, gọi ≤ 2 lần là đủ
- Không cần tăng cường độ campaign

---

### 3.2. Các segment conversion TRUNG BÌNH

| Segment | Conversion | Đặc trưng | Tên Business |
|-------|------------|-----------|--------------|
| 13 | 13.0% | Trẻ–trung niên, hành vi ổn định | **Standard Responsive Clients** |
| 15 | 12.1% | Gọi ít, phản hồi trung bình | **Neutral Potential Clients** |
| 18 | 11.6% | Lớn tuổi hơn, tương tác vừa | **Cautious Middle-aged Clients** |

**Insight:**
- Có thể khai thác nhưng không nên ưu tiên ngân sách lớn.

---

### 3.3. Các segment conversion THẤP (cảnh báo)

| Segment | Conversion | Đặc trưng | Tên Business |
|-------|------------|-----------|--------------|
| 21 | 10.0% | Thời lượng gọi rất ngắn | **Low-Interest Clients** |
| 14 | 9.6% | Trẻ nhưng ít quan tâm | **Hard-to-Engage Clients** |
| 12 | 9.5% | Gọi ít nhưng không hiệu quả | **Low-Return Prospects** |
| 23 | 9.3% | Trung niên, phản hồi thấp | **Low-Priority Clients** |

**Insight:**
- Gọi thêm không cải thiện conversion
- Nên giảm tần suất hoặc đổi kênh tiếp cận

## 5. Kết luận Business
- MiniBatchKMeans giúp hiểu **bức tranh tổng thể** hành vi khách hàng.
- HDBSCAN giúp phát hiện **nhóm khách hàng “vàng”** với conversion cao gấp ~2 lần trung bình.
- Segmentation bằng HDBSCAN là nền tảng tốt để:
  - Trích xuất luật (Decision Tree)
  - Thiết kế chiến lược campaign theo từng phân khúc
  - Tối ưu chi phí marketing

---

## 6. Hướng khai thác tiếp theo
- Tập trung rule extraction cho các segment:
  - **High-Potential Young Responders**
  - **Returning Interested Clients**
  - **Quick-Decision Young Clients**
- Xây dựng:
  - Pre-call rules (ai nên gọi)
  - In-call rules (đánh giá lead)


#### Rule Extraction theo segment bằng Decision Tree

Ta rút 2 loại luật:
1) Pre-call rules: không dùng duration (hữu ích để quyết định trước khi gọi)
2) In-call rules: có duration (lead scoring / đánh giá chất lượng cuộc gọi)

Mỗi segment → 1 cây nhỏ (max_depth thấp) để luật dễ hiểu.


In [27]:
def segment_rules(
    df,
    seg_col,
    features,
    *,
    target_col="y",
    max_depth=3,
    min_samples_leaf=80,
    min_n=500
):
    base = df[target_col].mean()
    rows = []

    for seg in sorted(df[seg_col].unique()):
        sub = df[df[seg_col] == seg]
        if len(sub) < min_n:
            rows.append({
                "segment": seg, "n": len(sub),
                "conversion": sub[target_col].mean(),
                "lift": sub[target_col].mean() / base if base > 0 else np.nan,
                "rules": "[SKIP] segment too small"
            })
            continue

        Xs = sub[features]
        ys = sub[target_col]

        tree = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        tree.fit(Xs, ys)

        rules = export_text(tree, feature_names=features, max_depth=max_depth)

        rows.append({
            "segment": seg,
            "n": len(sub),
            "conversion": float(ys.mean()),
            "lift": float(ys.mean() / base) if base > 0 else np.nan,
            "rules": rules
        })

    return pd.DataFrame(rows).sort_values("conversion", ascending=False), base


In [28]:
#quyết định nên tậpt trung gọi vô nhóm nào
FEATURES_PRE = ["age", "campaign", "previous"]

#đánh giá sau cuộc gọi
FEATURES_IN  = ["age", "campaign", "previous", "duration"]

rules_mbk_pre, base_rate = segment_rules(df, "segment_mbk", FEATURES_PRE, max_depth=3, min_samples_leaf=120)
rules_mbk_in, _          = segment_rules(df, "segment_mbk", FEATURES_IN,  max_depth=3, min_samples_leaf=120)

base_rate, rules_mbk_pre[["segment","n","conversion","lift"]]


(0.11698480458295547,
    segment      n  conversion      lift
 0        0  27747    0.120121  1.026809
 1        1  17464    0.112002  0.957405)

conversion chính là tỷ lệ chuyển đổi của khách hàng đồng ý/ tổng kh

Base rate = conversion trung bình của toàn bộ dữ liệu

lift = conversion / base_rate

- segment 0:

Segment này nhỉnh hơn trung bình, Nhưng không tạo ra khác biệt mạnh, Không đủ để xây dựng chiến lược riêng biệt

- segment 1: thấp hơn mặt bằng chung, k có gì wow

In [29]:
# In luật cho top 2 segment (pre-call)
for seg in rules_mbk_pre["segment"].head(2).tolist():
    row = rules_mbk_pre[rules_mbk_pre["segment"] == seg].iloc[0]
    print("="*90)
    print(f"MBK SEGMENT {seg} | n={row['n']} | conversion={row['conversion']:.3f} | lift={row['lift']:.2f}x")
    print(row["rules"])


MBK SEGMENT 0 | n=27747 | conversion=0.120 | lift=1.03x
|--- previous <= 0.50
|   |--- age <= 26.50
|   |   |--- age <= 22.50
|   |   |   |--- class: 0
|   |   |--- age >  22.50
|   |   |   |--- class: 0
|   |--- age >  26.50
|   |   |--- age <= 30.50
|   |   |   |--- class: 0
|   |   |--- age >  30.50
|   |   |   |--- class: 0
|--- previous >  0.50
|   |--- age <= 29.50
|   |   |--- age <= 24.50
|   |   |   |--- class: 0
|   |   |--- age >  24.50
|   |   |   |--- class: 0
|   |--- age >  29.50
|   |   |--- campaign <= 3.50
|   |   |   |--- class: 0
|   |   |--- campaign >  3.50
|   |   |   |--- class: 0

MBK SEGMENT 1 | n=17464 | conversion=0.112 | lift=0.96x
|--- age <= 60.50
|   |--- previous <= 0.50
|   |   |--- campaign <= 1.50
|   |   |   |--- class: 0
|   |   |--- campaign >  1.50
|   |   |   |--- class: 0
|   |--- previous >  0.50
|   |   |--- age <= 45.50
|   |   |   |--- class: 0
|   |   |--- age >  45.50
|   |   |   |--- class: 0
|--- age >  60.50
|   |--- previous <= 1.50
|

In [30]:
df_hdb_clean = df[df["segment_hdb"] != -1].copy()

rules_hdb_pre, base_rate_hdb = segment_rules(df_hdb_clean, "segment_hdb", FEATURES_PRE, max_depth=3, min_samples_leaf=120)
rules_hdb_in, _              = segment_rules(df_hdb_clean, "segment_hdb", FEATURES_IN,  max_depth=3, min_samples_leaf=120)

base_rate_hdb, rules_hdb_pre[["segment","n","conversion","lift"]].head(10)


(0.0853738882178544,
     segment     n  conversion      lift
 12       12  2664    0.182432  2.136865
 9         9  1952    0.177766  2.082210
 13       13  3472    0.126728  1.484390
 11       11  1543    0.123785  1.449914
 3         3  2810    0.114591  1.342222
 20       20  1167    0.107969  1.264662
 17       17  1200    0.105833  1.239645
 10       10   842    0.102138  1.196358
 16       16  2373    0.097345  1.140221
 0         0  1536    0.093099  1.090485)

In [31]:
# In luật cho top 2 cluster rõ nhất (pre-call)
for seg in rules_hdb_pre["segment"].head(2).tolist():
    row = rules_hdb_pre[rules_hdb_pre["segment"] == seg].iloc[0]
    print("="*90)
    print(f"HDB SEGMENT {seg} | n={row['n']} | conversion={row['conversion']:.3f} | lift={row['lift']:.2f}x")
    print(row["rules"])


HDB SEGMENT 12 | n=2664 | conversion=0.182 | lift=2.14x
|--- age <= 25.50
|   |--- age <= 22.50
|   |   |--- class: 0
|   |--- age >  22.50
|   |   |--- age <= 24.50
|   |   |   |--- class: 0
|   |   |--- age >  24.50
|   |   |   |--- class: 0
|--- age >  25.50
|   |--- previous <= 0.50
|   |   |--- campaign <= 1.50
|   |   |   |--- class: 0
|   |   |--- campaign >  1.50
|   |   |   |--- class: 0
|   |--- previous >  0.50
|   |   |--- age <= 30.50
|   |   |   |--- class: 0
|   |   |--- age >  30.50
|   |   |   |--- class: 0

HDB SEGMENT 9 | n=1952 | conversion=0.178 | lift=2.08x
|--- previous <= 0.50
|   |--- age <= 29.50
|   |   |--- age <= 27.50
|   |   |   |--- class: 0
|   |   |--- age >  27.50
|   |   |   |--- class: 0
|   |--- age >  29.50
|   |   |--- campaign <= 3.50
|   |   |   |--- class: 0
|   |   |--- campaign >  3.50
|   |   |   |--- class: 0
|--- previous >  0.50
|   |--- age <= 31.50
|   |   |--- class: 0
|   |--- age >  31.50
|   |   |--- age <= 34.50
|   |   |   |--- c